### Code

In [1]:
import numpy as np
import pandas as pd
import os
from utils.case_directory import CaseDirectory
from utils.case_metadata import CaseMetadata
from extractors.jury_ruling_classifier import JuryRulingClassifier

In [10]:
def summarize_trial_classification():
    d = CaseDirectory("100_random_sample")
    d.categorize_cases("random_categories.csv")
    df = pd.read_csv("random_categories.csv")
    df2 = pd.read_csv("labeled_cases.csv")
    print("Percentage keyword cases correctly classified")
    print(len(df2[(df.trial == ((df2.trial_type == "jury") | (df2.trial_type == "bench")))])/len(df2))
    print("Categorization of cases misclassified")
    print(df2[(df.trial != ((df2.trial_type == "jury") | (df2.trial_type == "bench")))].trial_type.value_counts())

### Find Trial Docs

In [2]:
directory = CaseDirectory("100_random_fed")

In [3]:
directory.categorize_cases("fed_categories.csv")

In [4]:
df = pd.read_csv("fed_categories.csv")

In [6]:
trial_docs = {}

In [12]:
trial_paths = df[df.trial == True].metadata_path.tolist()

In [115]:
path = trial_paths[29]
case = CaseMetadata.from_metadata_path(path)
dr = case.get_docket_report()
dr[dr.document_path != ""]

,link_viewer,number,exhibits,link,entry_date,date,contents,document_path
2,https://www.docketalarm.com/cases/Illinois_Cen...,481,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2011-04-18,ORDER & OPINION entered by Judge Joe Billy Mc...,100_random_fed/Illinois_Central_District_Court...
3,https://www.docketalarm.com/cases/Illinois_Cen...,481,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2011-04-18,ORDER & OPINION entered by Judge Joe Billy Mc...,100_random_fed/Illinois_Central_District_Court...
6,https://www.docketalarm.com/cases/Illinois_Cen...,478,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2010-07-13,ORDER entered by Magistrate Judge John A. Gorm...,100_random_fed/Illinois_Central_District_Court...
36,https://www.docketalarm.com/cases/Illinois_Cen...,448,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2009-12-15,ORDER entered by Judge Joe Billy McDade on 12...,100_random_fed/Illinois_Central_District_Court...
37,https://www.docketalarm.com/cases/Illinois_Cen...,448,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2009-12-15,ORDER entered by Judge Joe Billy McDade on 12...,100_random_fed/Illinois_Central_District_Court...
99,https://www.docketalarm.com/cases/Illinois_Cen...,386,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-09-06,ORDER granting in part and denying in part 38...,100_random_fed/Illinois_Central_District_Court...
289,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...
327,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...
365,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 MOTI...,100_random_fed/Illinois_Central_District_Court...
403,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...


In [111]:
title = dr.loc[752, "contents"]
doc = case.get_document_by_docket_report_title(title)
print(list(doc.values())[0])

1:04-cv-01118-JBM-JAG   # 93    Page 1 of 3                                              

E-FILEDE-FILEDE-FILED
     

 Monday, 26 September, 2005  02:49:50 PM  Monday, 26 September, 2005  02:50:25 PM  Monday, 26 September, 2005  02:51:00 PM 

 Clerk, U.S. District Court, ILCD Clerk, U.S. District Court, ILCD Clerk, U.S. District Court, ILCD
UNITED STATES DISTRICT COURT
CENTRAL DISTRICT OF ILLINOIS
TRISH LEE McCLOUD, by
)
and through her legal
)
guardian, Candy L. Hall,
Plaintiff,
))
))
)          Case No. 04-1118
v.
)
GOODYEAR DUNLOP TIRES NORTH )
AMERICA, LTD. and THE         )
GOODYEAR TIRE & RUBBER    
)
COMPANY, 
))
)
Defendants.
O R D E R
In separate lawsuits Plaintiff Trish Lee McCloud, a passenger
on a motorcycle being operated by William Booker (Case No. 04-1118)
and Plaintiff William Booker (Case No. 04-1159) sued Defendants
Goodyear Dunlop Tires North America, Ltd. (Dunlop) and The Goodyear
Tire & Rubber Company (Goodyear) (collectively referred to as
Dunlop/Goodyear) for i

In [112]:
trial_docs.update(doc)

In [113]:
trial_docs.keys()

dict_keys(['100_random_fed/Texas_Eastern_District_Court/2--21-cv-00084/The_Chamberlain_Group_LLC_v._Overhead_Door_Corporation_et_al/600.txt', '100_random_fed/Wisconsin_Western_District_Court/3--14-cv-00099/Haley_Mary_et_al_v._Kolbe_and_Kolbe_Millwork_Co._Inc._et_al/603.txt', '100_random_fed/California_Northern_District_Court/5--16-cv-01370/Sumotext_Corp._-v-_Zoove_Inc._et_al/470.txt', '100_random_fed/New_Jersey_District_Court/1--16-cv-00395/KUHAR_et_al_v._PETZL_COMPANY_et_al/234.txt', '100_random_fed/New_York_Southern_District_Court/1--10-cv-02881/The_Estate_of_Mauricio_Jaquez_v._The_City_of_New_York_et_al/186.txt', '100_random_fed/North_Carolina_Western_District_Court/3--15-cv-00309/Poole_v._Gaston_County_et_al/134.txt', '100_random_fed/California_Central_District_Court/2--06-cv-03923/Aleksandr_L_Yufa_v._Lockheed_Martin_Corporation_et_al/311.txt', '100_random_fed/California_Central_District_Court/5--15-cv-01921/Bennion_and_Deville_Fine_Homes_Inc_et_al_v._Windermere_Real_Estate_Service

In [116]:
d = [{"path": path, "text": text} for (path, text) in trial_docs.items()]

In [117]:
pd.DataFrame(d).to_csv("federal_trial_docs.csv", index=False)

In [118]:
df2 = pd.read_csv("federal_trial_docs.csv")

In [121]:
df2.path.tolist()

['100_random_fed/Texas_Eastern_District_Court/2--21-cv-00084/The_Chamberlain_Group_LLC_v._Overhead_Door_Corporation_et_al/600.txt',
 '100_random_fed/Wisconsin_Western_District_Court/3--14-cv-00099/Haley_Mary_et_al_v._Kolbe_and_Kolbe_Millwork_Co._Inc._et_al/603.txt',
 '100_random_fed/California_Northern_District_Court/5--16-cv-01370/Sumotext_Corp._-v-_Zoove_Inc._et_al/470.txt',
 '100_random_fed/New_Jersey_District_Court/1--16-cv-00395/KUHAR_et_al_v._PETZL_COMPANY_et_al/234.txt',
 '100_random_fed/New_York_Southern_District_Court/1--10-cv-02881/The_Estate_of_Mauricio_Jaquez_v._The_City_of_New_York_et_al/186.txt',
 '100_random_fed/North_Carolina_Western_District_Court/3--15-cv-00309/Poole_v._Gaston_County_et_al/134.txt',
 '100_random_fed/California_Central_District_Court/2--06-cv-03923/Aleksandr_L_Yufa_v._Lockheed_Martin_Corporation_et_al/311.txt',
 '100_random_fed/California_Central_District_Court/5--15-cv-01921/Bennion_and_Deville_Fine_Homes_Inc_et_al_v._Windermere_Real_Estate_Services_C

### Classification

In [124]:
path = "100_random_sample/New_York_State_Suffolk_County_Supreme_Court/602235---2016/metadata.json"
classifier = JuryRulingClassifier(path, language_model="llama3.1", llm_document_count=7)

In [10]:
classifier.extract()

Extracting from metadata...
- Getting relevant chunks...
- Querying llm...
- Response: {'reasoning': "The documents provided are all related to the plaintiffs' proposed verdict sheets and jury instructions, which suggests that the case is focused on the plaintiff's claims. However, there is no indication of a jury verdict or a ruling in favor of either party. The lack of reference to a verdict or a specific outcome indicates that the decision of the jury trial is not identified in these documents.", 'category': 'undetermined'}
Extracting from documents...
- Getting relevant chunks...
- Querying llm...
- Response: {'reasoning': "The jury verdict forms do not explicitly identify a final decision regarding the plaintiff's claim against the defendant. However, Verdict 4 asks for damages, which implies that the plaintiffs have been found liable by the jury. This shows that the jury ruled in favor of the plaintiffs because they were awarded damages.", 'category': 'plaintiff'}


{'reasoning': "The jury verdict forms do not explicitly identify a final decision regarding the plaintiff's claim against the defendant. However, Verdict 4 asks for damages, which implies that the plaintiffs have been found liable by the jury. This shows that the jury ruled in favor of the plaintiffs because they were awarded damages.",
 'category': 'plaintiff'}

In [133]:
prompt = """
        You are an expert legal analyst. You will be given a sequence of indices and legal summaries from legal documents relating to a case in the United States. Documents are separated by ||. All documents correspond to the same case. 
        Based on the summaries, determine which documents may contain the outcome of the case.
        Relevant documents may include decision, verdict, opinion, sentencing, and trial documents. 
        Respond with a JSON object in the form {"reasoning": "...", "indices": ...}
        reasoning should be in the format "The summary of document _ contains _, so document _ is likely to contain the outcome of the case because _"
        indices should be a list of the indices of relevant documents. Do not include anything other than the relevant indices in indices. 
        """

In [134]:
docs = classifier.metadata.get_docket_report_contents()

In [135]:
s = ""
for i, doc in enumerate(docs):
    s += (f"{i}: {doc}||")

In [136]:
s

"0: DECISION + ORDER ON MOTION (Motion #004)||1: AFFIDAVIT OR AFFIRMATION IN OPPOSITION TO CROSS-MOTION AND IN FURTHER SUPPORT OF MOTION (Motion #001)||2: EXHIBIT(S) - 1 (Motion #004) Exhibit 1 - First Midwest South Willowbrook Statements||3: AFFIDAVIT OR AFFIRMATION IN OPPOSITION TO MOTION (Motion #004) Affirmation in Opposition||4: EXTRACT Trial Extract||5: OTHER COURT FILED DOCUMENT Exhibit Sheet||6: VERDICT SHEET||7: JUROR NOTE(S)||8: JUROR NOTE(S)||9: EXHIBIT(S) Court Exhibit II / Statement of Claim||10: MARKED PLEADINGS Court Exhibit I||11: ADMISSION OF SERVICE (Motion #004)||12: AFFIDAVIT OR AFFIRMATION IN SUPPORT OF MOTION (Motion #004)||13: NOTICE OF MOTION (Motion #004)||14: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||15: TRIAL DOCUMENTS Plaintiffs' Proposed Jury Instructions||16: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||17: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||18: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||19: TRIAL DOCUMENTS Pl

In [138]:
import ollama

In [139]:
response = ollama.generate(
            model="llama3.1",
            prompt=s,
            system=prompt,
            format="json",
        )

KeyboardInterrupt: 

In [ ]:
response

### To do
1. Make it easier to visualize jury trials/extract jury trial results and do the same for other variables
2. Add reranking algorithm
3. Add helper function to manually label

- Designed application to extract variables from over 15 million legal documents using natural language processing methods
- Developed over 5,000 lines of maintainable and documented 
- Coordinated meetings between research staff and legal librarian, presented variable extraction approach to research group

Reranking
- https://adasci.org/a-hands-on-guide-to-enhance-rag-with-re-ranking/
- https://techcommunity.microsoft.com/t5/microsoft-developer-community/doing-rag-vector-search-is-not-enough/ba-p/4161073
- https://community.openai.com/t/bad-formats-for-semantic-search-of-rag-implementing-internal-chatbot-for-troubleshooting-an-sdk/848715
- https://learn.microsoft.com/en-us/azure/search/index-similarity-and-scoring
- https://cohere.com/blog/rerank-3
- https://www.reddit.com/r/LocalLLaMA/comments/1d9h2pg/doing_rag_vector_search_is_not_enough/
- https://www.datacamp.com/tutorial/boost-llm-accuracy-retrieval-augmented-generation-rag-reranking
- https://python.langchain.com/v0.2/docs/integrations/retrievers/flashrank-reranker/

RAG
- https://ollama.com/blog/embedding-models
- https://huggingface.co/learn/nlp-course/chapter5/6
- https://docs.mistral.ai/guides/rag/
- https://docs.trychroma.com/guides